In [1]:
import pandas as pd
df = pd.read_csv("results_summarised/sort_different_depth_17_05_2.csv")

In [2]:
# convert test_loss_list into two columns epoch and test_loss
import ast
import pandas as pd

# Assuming df is your DataFrame and 'test_loss_list' is the column you're working with.

for index, row in df.iterrows():
    try:
        # Try to evaluate the row as a literal.
        pd.eval(row['test_loss_list'])
    except Exception as e:
        # If an exception occurs, print the index and the value that failed
        print(f"Failed at index {index}: {row['test_loss_list']}")
        print(f"Error: {e}")
        # Optionally, break after the first failure
        #break
import ast
# replace "nan" with None
df['test_loss_list'] = df['test_loss_list'].str.replace('nan', 'None')
df['loss_list'] = df['loss_list'].str.replace('nan', 'None')
#df['test_loss_list'] = df['test_loss_list'].apply(lambda s: pd.eval(s))
#df['loss_list'] = df['loss_list'].apply(lambda s: pd.eval(s))

df['test_loss_list'] = df['test_loss_list'].apply(ast.literal_eval)
df['loss_list'] = df['loss_list'].apply(ast.literal_eval)

# Create a new DataFrame for Plotly Express
rows_list = []
for _, row in df.iterrows():
    params = row.to_dict()
    test_loss_list = params.pop('test_loss_list')
    loss_list = params.pop('loss_list')
    for epoch, loss in enumerate(test_loss_list):
        params['epoch'] = epoch
        params['test_loss'] = loss
        params['train_loss'] = loss_list[epoch]
        rows_list.append(params.copy())  # Use .copy() to avoid modifying the original params

# Create a new DataFrame
df_long = pd.DataFrame(rows_list)
df_long = df_long.fillna("missing")

In [3]:
df_long = df_long.fillna("missing")
# convert all to string except test_loss
# removing rows where test_loss is missing
df_long = df_long[df_long["test_loss"] != "missing"]
# same for epoch
df_long = df_long[df_long["epoch"] != "missing"]
df_long = df_long.applymap(str)
df_long["test_loss"] = df_long["test_loss"].astype(float)
df_long["epoch"] = df_long["epoch"].astype(int)
df_long["train_loss"] = df_long["train_loss"].astype(float)

# drop 	Unnamed: 0
def convert_embedding(name):
    if "GaussianEmbedder" in name:
        return "gaussian"
    elif "PositionalEncoding" in name:
        return "positional"
    elif "RandomFeature" in name:
        return "fourier"
    else:
        return "none"
df_long["embedding"] = df_long["embedding_function"].apply(convert_embedding)

# remove Unnamed: 0
df_long = df_long.drop(columns=["Unnamed: 0", "job_id"])

/var/folders/fy/__8z8cpn6gs04465sq9g1nq80000gn/T/ipykernel_64917/2401676600.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_long = df_long.applymap(str)


In [4]:
# group
group_cols = list(df_long.columns.difference(['test_loss', 'train_loss', "iter"]))
df_grouped = df_long.groupby(group_cols)['test_loss'].mean().reset_index()

In [27]:
import plotly.express as px
import numpy as np
import plotly.graph_objects as go

df_plot = df_long.copy()
df_plot["epoch"] = df_plot["epoch"].astype(str)

# compute baseline
filter_dic_baseline = {
    "n_layer": "2",
    "n_embd": "256",
    #"embedding": "gaussian",
    "read_in_bias": "False",
    "epoch": 299,
}

df_baseline = df_plot.copy()
for key, value in filter_dic_baseline.items():
    df_baseline = df_baseline[np.isin(df_baseline[key], [value, "missing"])]

# only fourier or gaussian
df_baseline = df_baseline[df_baseline["embedding"] != "none"]

# compute the mean test_loss grouped by embedding
df_baseline_grouped = df_baseline.groupby(["embedding", "epoch"])["test_loss"].median().reset_index()






filter_dic = {
    "epoch": "299",
    #"n_embd": "256",
    "embedding": "none",
    "read_in_bias": "False",
    "attention_only": "True",
    "n_epochs": "400",
}
# remove 128 n_embd
df_plot = df_plot[df_plot["n_embd"] != "128"]

#df_plot = df_long.copy()
for key, value in filter_dic.items():
    df_plot = df_plot[np.isin(df_plot[key], [value, "missing"])]

df_plot["n_layer"] = df_plot["n_layer"].astype("int")
# order n_layer
#df_plot["n_layer"] = pd.Categorical(df_plot["n_layer"], categories=["1", "2", "4", "6", "8", "10", "12", "14", "16", "18", "20"], ordered=True)
# order n_embd as categorical
df_plot["n_embd"] = df_plot["n_embd"].astype(str)
#df_plot["n_embd"] = pd.Categorical(df_plot["n_embd"], categories=["128", "256", "512"], ordered=True)

fig = px.box(df_plot, 
           x="n_layer", 
           #y="embedding", 
           y = "test_loss",
           color="n_embd", 
           hover_data=["iter", "n_embd"],
           title="Test Loss vs. Epoch",
           #facet_col="read_in_bias",
           #facet_row="embedding",
           log_y=True,
           category_orders={"n_layer": ["1", "2", "4", "8", "16", "32"]}, # removed 1
           log_x=True,
           height=600,
)

# print count per n_layer
print(df_plot["n_layer"].value_counts())
print(df_plot["attention_only"].value_counts())
print(df_plot["n_epochs"].value_counts())



# # Add horizontal lines and annotations for the baseline
for i, embedding in enumerate(df_baseline["embedding"].unique()):
    baseline_data = df_baseline_grouped[df_baseline_grouped["embedding"] == embedding]
    color = "grey" if i else "black"
    for i, row in baseline_data.iterrows():
        fig.add_hline(y=row["test_loss"], line_dash="dash", line_color=color)
        # Adjust the annotation position to be near the right side of the plot
        fig.add_annotation(
            x=0.95,  # Use relative coordinates (0 to 1) for x position
            y=row["test_loss"], 
            #text=f"{embedding} baseline (epoch {row['epoch']})", 
            showarrow=True, 
            arrowhead=2,
            ax=0.95,  # Position arrow base at the same relative x position
            ay=row["test_loss"],  # Position arrow base at the same y value
            xref="paper",  # Reference x to the entire plot
            yref="y",  # Reference y to the y-axis value
            font=dict(size=14),
            #bgcolor="white",
            #bordercolor="black",
            borderwidth=2,
        )

# Create baseline box plots and add them to the main plot
# Create baseline box plots and add them to the main plot
for i, embedding in enumerate(df_baseline["embedding"].unique()):
    df_baseline_embedding = df_baseline[df_baseline["embedding"] == embedding]
    increment = 0.4 if i else -0.4  # Shift the baseline box plots to the left or right
    df_baseline_embedding["n_layer"] = df_baseline_embedding["n_layer"].astype(float) + increment
    color = "grey" if i else "black"
    baseline_fig = px.box(
        df_baseline_embedding, 
        x="n_layer", 
        y="test_loss",
        color="n_embd",
        color_discrete_sequence=[color],  # Set to black
        log_y=True,
        height=600,
    )
    for trace in baseline_fig.data:
        trace.update(showlegend=False)  # Hide legend for the baseline traces
        fig.add_trace(trace)

    
# Update layout for better readability
fig.update_layout(
    #title="Test Loss vs. Epoch",
    title="",
    xaxis_title="Number of Layers",
    yaxis_title="Test MSE",
    legend_title="Embedding Dimension",
    font=dict(
        family="Arial, sans-serif",
        size=18,  # Increase the font size
        color="black"
    ),
    title_font=dict(
        size=20,  # Increase the title font size
        color="black"
    ),
    legend=dict(
        x=1.05,
        y=1,
        traceorder="normal",
        bgcolor="white",
        bordercolor="black",
        borderwidth=1,
        font=dict(size=14),  # Increase legend font size
    ),
    margin=dict(l=40, r=40, t=60, b=40),  # Adjust margins for better layout
    # make box wider
    boxmode='overlay',
    boxgap=0.2,
    boxgroupgap=0.2,
)
# make ticks only for the layers in the plot
fig.update_xaxes(tickvals=[1, 2, 4, 8, 16, 32])
# make the plot square
fig.update_layout(height=600, width=800)

fig.show()

# save to pdf
import plotly.io as pio
#pio.write_image(fig, "plots/sort_n_layers.pdf")

# save to svg
import plotly.io as pio
pio.write_image(fig, "plots/sort_n_layers_2.svg")

n_layer
8     10
1     10
4     10
16    10
2     10
32     5
Name: count, dtype: int64
attention_only
True    55
Name: count, dtype: int64
n_epochs
400    55
Name: count, dtype: int64


/var/folders/fy/__8z8cpn6gs04465sq9g1nq80000gn/T/ipykernel_64917/1036408037.py:104: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/fy/__8z8cpn6gs04465sq9g1nq80000gn/T/ipykernel_64917/1036408037.py:104: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

